In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Conv2d - 32, ReLU, Normalization, MaxPool
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # Input: [-1, 1, 28, 28], Output: [-1, 64, 28, 28]
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # Output: [-1, 32, 14, 14]
        self.dropout1 = nn.Dropout(0.05)

        # Conv2d - 32, ReLU, Normalization, MaxPool
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)  # Input: [-1, 32, 14, 14], Output: [-1, 32, 14, 14]
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)  # Output: [-1, 32, 7, 7]
        #self.dropout2 = nn.Dropout(0.05)

        # Conv2d - 20, ReLU, Normalization, MaxPool
        self.conv3 = nn.Conv2d(32, 20, 3, padding=1)  # Input: [-1, 32, 7, 7], Output: [-1, 20, 7, 7]
        self.batch_norm3 = nn.BatchNorm2d(20)
        #self.pool3 = nn.MaxPool2d(2, 2)  # Output: [-1, 20, 3, 3]
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Conv2d - 10, ReLU, Normalization, MaxPool
        self.conv4 = nn.Conv2d(20, 10, 3, padding=1)  # Input: [-1, 20, 3, 3], Output: [-1, 10, 3, 3]
        self.batch_norm4 = nn.BatchNorm2d(10)
        #self.pool4 = nn.MaxPool2d(2, 2)  # Output: [-1, 10, 1, 1]
        self.global_avg_pool = nn.AvgPool2d(kernel_size=3)  # Output: [-1, 10, 1, 1]

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = F.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = F.max_pool2d(x, 2)
        #x = self.dropout2(x)

        x = F.relu(self.conv3(x))
        x = self.batch_norm3(x)
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv4(x))
        x = self.batch_norm4(x)
        #x = F.max_pool2d(x, 2)
        x = self.global_avg_pool(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
       BatchNorm2d-5           [-1, 32, 14, 14]              64
            Conv2d-6             [-1, 20, 7, 7]           5,780
       BatchNorm2d-7             [-1, 20, 7, 7]              40
            Conv2d-8             [-1, 10, 3, 3]           1,810
       BatchNorm2d-9             [-1, 10, 3, 3]              20
        AvgPool2d-10             [-1, 10, 1, 1]               0
Total params: 17,346
Trainable params: 17,346
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.54
Params size (MB): 0.07
Estimated Tot

<ipython-input-3-78949241671e>:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #transforms.RandomRotation((-15., 15.), fill=0),
                        transforms.RandomRotation(10),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 110003204.77it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 30620751.72it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27496606.19it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12968365.40it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-3-78949241671e>:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.1726083755493164 batch_id=1874: 100%|██████████| 1875/1875 [01:51<00:00, 16.85it/s]



Test set: Average loss: 0.0543, Accuracy: 9837/10000 (98.37%)



loss=0.021478872746229172 batch_id=1874: 100%|██████████| 1875/1875 [01:49<00:00, 17.05it/s]



Test set: Average loss: 0.0394, Accuracy: 9876/10000 (98.76%)



loss=0.007688732352107763 batch_id=1874: 100%|██████████| 1875/1875 [01:56<00:00, 16.04it/s]


...


Test set: Average loss: 0.0216, Accuracy: 9930/10000 (99.30%)



loss=0.04794040694832802 batch_id=1874: 100%|██████████| 1875/1875 [01:47<00:00, 17.48it/s]



Test set: Average loss: 0.0215, Accuracy: 9933/10000 (99.33%)



loss=0.0807574987411499 batch_id=1874: 100%|██████████| 1875/1875 [02:00<00:00, 15.51it/s]



Test set: Average loss: 0.0220, Accuracy: 9940/10000 (99.40%)

